In [93]:
print("Hello Neemo !")

Hello Neemo !


In [94]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [95]:
df = pd.read_csv(r'D:\IMP  ML  PROJECTS\CAR PRICE PREDICTION\web scraping\MUMBAI\feature engineering\cleaned_car_data_Petrol.csv')
print(df)

      Registration Year  Insurance  Seats  Kms Driven  Ownership  \
0                2022.0          0    5.0       30000          1   
1                2024.0          2    7.0        8000          1   
2                2024.0          0    7.0       20277          1   
3                2022.0          2    5.0       20000          1   
4                2023.0          2    5.0       27802          1   
...                 ...        ...    ...         ...        ...   
3746             2018.0          0    5.0       44901          1   
3747             2016.0          1    5.0       79584          2   
3748             2017.0          2    5.0       65834          2   
3749             2016.0          1    5.0       73026          1   
3750             2020.0          2    5.0       17752          1   

      Engine Displacement  Mileage  Max_Power_clean  Max_Power_RPM  \
0                     999    17.20           113.98         5500.0   
1                    1451    12.34         

In [96]:
# Edge cases
for col in df.columns:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])

In [97]:
df.columns

Index(['Registration Year', 'Insurance', 'Seats', 'Kms Driven', 'Ownership',
       'Engine Displacement', 'Mileage', 'Max_Power_clean', 'Max_Power_RPM',
       'Max_Torque_Nm', 'Max_Torque_RPM', 'Valves Per Cylinder',
       'Turbo Charger', 'Transmission Type', 'Gearbox', 'Petrol Mileage ARAI',
       'Petrol Fuel Tank Capacity', 'Emission Norm Compliance',
       'Front Suspension', 'Rear Suspension', 'Steering Type',
       'Steering Column', 'Front Brake Type', 'Rear Brake Type',
       'Ground Clearance Unladen', 'Wheel Base', 'Price', 'Top Speed',
       'Volume', 'Drive Type_4WD', 'Drive Type_AWD', 'Drive Type_FWD',
       'Drive Type_RWD', 'Engine Type_Standard',
       'Engine Type_Standard Brand Engine', 'Engine Type_Turbocharged',
       'Engine Type_V-Type', 'Fuel Supply System_Direct Injection',
       'Fuel Supply System_EFI', 'Fuel Supply System_MPFi',
       'Fuel Supply System_Other', 'Gross Weight'],
      dtype='object')

In [98]:
numerical_cols = []
categorical_cols = []

for col in df.columns:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        numerical_cols.append(col)
    else :
        categorical_cols.append(col)
        
print(numerical_cols)
print(categorical_cols)

['Registration Year', 'Insurance', 'Seats', 'Kms Driven', 'Ownership', 'Engine Displacement', 'Mileage', 'Max_Power_clean', 'Max_Power_RPM', 'Max_Torque_Nm', 'Max_Torque_RPM', 'Valves Per Cylinder', 'Turbo Charger', 'Transmission Type', 'Gearbox', 'Petrol Mileage ARAI', 'Petrol Fuel Tank Capacity', 'Emission Norm Compliance', 'Front Suspension', 'Rear Suspension', 'Steering Type', 'Steering Column', 'Front Brake Type', 'Rear Brake Type', 'Ground Clearance Unladen', 'Wheel Base', 'Price', 'Top Speed', 'Volume', 'Drive Type_4WD', 'Drive Type_AWD', 'Drive Type_FWD', 'Drive Type_RWD', 'Engine Type_Standard', 'Engine Type_Standard Brand Engine', 'Engine Type_Turbocharged', 'Engine Type_V-Type', 'Fuel Supply System_Direct Injection', 'Fuel Supply System_EFI', 'Fuel Supply System_MPFi', 'Fuel Supply System_Other', 'Gross Weight']
[]


In [99]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Price', axis=1), df['Price'], test_size=0.2, random_state=42)

In [100]:
param_grid = {
    # Number of trees: 500 is usually plenty for 5k rows
    'n_estimators': [100, 300, 500], 
    
    # Depth: Keep it shallow (3-5) to avoid overfitting small data
    'max_depth': [3, 4, 5], 
    
    # Learning rate: Smaller is better for stability
    'learning_rate': [0.01, 0.05, 0.1], 
    
    # Regularization: Very important for 43 features on 5k rows
    'gamma': [0, 0.1, 0.2],
    
    # Subsampling: Only use 80% of data/features per tree to increase variety
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

In [101]:
xgb_model = xgb.XGBRegressor(tree_method='hist', random_state=42)

grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1, # Uses all your CPU cores for speed
    verbose=1
)

In [102]:
grid_search.fit(X_train, y_train)

# 5. Extract the best model
best_xgb_model = grid_search.best_estimator_

print("Best Parameters Found:", grid_search.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


Best Parameters Found: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}


In [103]:
# 1. Use the best model found by GridSearch to make predictions
y_pred = best_xgb_model.predict(X_test)

# 2. Calculate Evaluation Metrics
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 3. Print the results
print("--- Model Performance ---")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R2 Score (Accuracy): {r2:.4%}")

--- Model Performance ---
Mean Absolute Error: 507276.37
R2 Score (Accuracy): 1.5455%
